<a href="https://colab.research.google.com/github/PhucVinhDEV/Chat_BOT_DB/blob/main/Chatbot_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai chromadb gradio langchain-google-genai

In [47]:
sql_path= '/content/drive/MyDrive/ChatBot/products.db'

In [63]:
import os
import getpass

os.environ['GOOGLE_API_KEY'] = getpass.getpass('Google API Key:')

Google API Key:··········


In [64]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"sqlite:///{sql_path}")

print(db.dialect)
print(db.get_usable_table_names())
db.run("select * from products limit 10")

sqlite
['categories', 'products']


"[('hoanghamobile_626_2115_hoanghamobile.com', 'Ổ cắm điện thông minh chống giật Vconnex Smart Plug - Chính hãng', 'Ổ Cắm điện|Phụ kiện', 13, 603625.9, 96), ('lazadacps_476_903850875_lazada.vn', 'Nước hoa hồng Meishoku Bigansui Medicated Skin Lotion 200ml Nhật bản', 'Nước hoa hồng Meishoku Bigansui Medicated Skin Lotion 200ml Nhật BảnMeishoku Bigansui Medicated Skin Lotion 200ml - Là sản phẩm được sản xuất từ năm 1885 và cho đến ngày nay nó vẫn là một trong những sản phẩm trị mụn rất được phụ nữ Nhật Bản yêu thích và lựa chọn. Meishoku Bigansui Medicated...', 17, 747068.0, 42), ('lazadacps_476_6148238297_lazada.vn', 'Tã Quần Người Già Caryn Mỏng Nhẹ Chống Tràn Dành Cho Người Có Thể Đi Lại Được XL12', 'Tã Quần Người Lớn Caryn Mỏng Nhẹ Chống Tràn Dành Cho Người Già Có Thể Tự Đi Lại Được Tã quần mỏng nhẹ Caryn với thiết kế như quần lót, giúp người dùng dễ dàng tự mặc/thay tã và cho cảm giác thoải mái khi vận động. Sản phẩm phù hợp với những người gặp vấn đề về bài tiết nhưng vẫn có thể tự

In [65]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", temperature=0)

## Few shot


**Example Query**

In [66]:
examples = [
    {
        "input" : "Danh mục sản phẩm có những gì",
        "query" : """
          select id,name from categories
        """
    }
]

In [67]:
from langchain.embeddings import HuggingFaceEmbeddings

# 2. Khởi tạo mô hình embedding miễn phí từ HuggingFace

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [68]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

# 3. Tạo example selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=embedding,  # ✅ đúng tên tham số
    vectorstore_cls=Chroma,
    k=1,
    input_keys=['input'],
)

In [69]:
example_selector.select_examples({"input" : "Sản phẩm nào dưới 300 nghìn"})

[{'query': '\n          select id,name from categories\n        ',
  'input': 'Danh mục sản phẩm có những gì'}]

## Correct productName


In [70]:
from langchain.agents.agent_toolkits import create_retriever_tool

In [71]:
import ast


def query_as_list(db, query):
  res = db.run(query)
  res = [el for sub in ast.literal_eval(res) for el in sub if el]
  return list(set(res))

product = query_as_list(db, "select name from products") + query_as_list(db, "select name from categories")
print(product[:-10])

['[Mã TOYDEC hoàn 20K xu đơn 50K] Lăp ráp xếp hình Lego city 23002 : xe cần cẩu móc master builder 465 mảnh', 'Loa Soundmax A140 2.0 Bạc - Thiết kế nhỏ gọn, năng động - Hàng chính hãng', 'Áo Thun In Chữ TĂNG HIGH TĂNG BAR KHÔNG TĂNG CA Meme Hài Hước Unisex Nam Nữ UT', 'Giày lười da nam công sợ Bụi Leather G121 - Da bò Nappa cao cấp - 40 - Màu đen  Bộ vệ sinh', 'muối sữa spa bonne', 'Trục giảm tốc HONDA Sh 125 / 150 (2012-2016)  QA(23430KZR600)', 'Dây ga SH 125/150  QA(17910K01901) _( 6B 3A)', 'Nước hoa hồng Meishoku Bigansui Medicated Skin Lotion 200ml Nhật bản', 'Nắp đậy nhôm pô tăng xe đạp PRO S-WORKS ENVE ZIPP KOCEVLO RITCHEY (7 Màu)', 'Đèn bàn khử trùng Philips UV-C', 'Tã Quần Người Già Caryn Mỏng Nhẹ Chống Tràn Dành Cho Người Có Thể Đi Lại Được XL12', 'Ổ cắm điện thông minh chống giật Vconnex Smart Plug - Chính hãng', '❤️[Tóc Giả Loại 1] ❤️- Tóc Cả Đầu Layer Trending', 'FREE SHIP DÂY CHUYỀN NỮ ĐANG SIÊU HÓT BẠC TA', 'Bộ đồ bầu ngố mặc nhà chất lanh mát mùa hè nhiều vân', 'Bộ 2 cút

## Correct Product Name

In [72]:
from langchain.agents.agent_toolkits import create_retriever_tool

In [73]:
import ast

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    return list(set(res))

product = query_as_list(db, "select name from products") + query_as_list(db, "select name from categories")
print(product[:-10])

['[Mã TOYDEC hoàn 20K xu đơn 50K] Lăp ráp xếp hình Lego city 23002 : xe cần cẩu móc master builder 465 mảnh', 'Loa Soundmax A140 2.0 Bạc - Thiết kế nhỏ gọn, năng động - Hàng chính hãng', 'Áo Thun In Chữ TĂNG HIGH TĂNG BAR KHÔNG TĂNG CA Meme Hài Hước Unisex Nam Nữ UT', 'Giày lười da nam công sợ Bụi Leather G121 - Da bò Nappa cao cấp - 40 - Màu đen  Bộ vệ sinh', 'muối sữa spa bonne', 'Trục giảm tốc HONDA Sh 125 / 150 (2012-2016)  QA(23430KZR600)', 'Dây ga SH 125/150  QA(17910K01901) _( 6B 3A)', 'Nước hoa hồng Meishoku Bigansui Medicated Skin Lotion 200ml Nhật bản', 'Nắp đậy nhôm pô tăng xe đạp PRO S-WORKS ENVE ZIPP KOCEVLO RITCHEY (7 Màu)', 'Đèn bàn khử trùng Philips UV-C', 'Tã Quần Người Già Caryn Mỏng Nhẹ Chống Tràn Dành Cho Người Có Thể Đi Lại Được XL12', 'Ổ cắm điện thông minh chống giật Vconnex Smart Plug - Chính hãng', '❤️[Tóc Giả Loại 1] ❤️- Tóc Cả Đầu Layer Trending', 'FREE SHIP DÂY CHUYỀN NỮ ĐANG SIÊU HÓT BẠC TA', 'Bộ đồ bầu ngố mặc nhà chất lanh mát mùa hè nhiều vân', 'Bộ 2 cút

In [74]:
vector_db = Chroma.from_texts(product, embedding)
retriever = vector_db.as_retriever(search_kwargs={"k":15})
description = "Use to look up values to filter on. Input is an approxiamte spelling of the proper noun, output is valid proper nouns. Use the noun most similar to the search"
retriever_tool = create_retriever_tool(
    retriever,
    description=description,
    name="search_proper_nouns"
)

In [75]:
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate, MessagesPlaceholder, PromptTemplate


In [76]:
def get_answer(user_query):
  top_k = 20
  dialect = db.dialect

  system_prefix = f"""
  You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:
"""
  # print(system_prefix)
  ##fall_back
  fallback_example = [
    {
        "input": "Danh mục sản phẩm có những gì?",
        "query": """
        select id, name from categories
        """
    }
  ]

  try:
    safe_examples = example_selector.select_examples({"input" : user_query})
    example_prompt = PromptTemplate.from_template("User input: {input}\n SQL query: {query}")
    for ex in safe_examples:
      if "input" not in ex or "query" not in ex:
        raise ValueError("Every example must contain an input and query key.")
    few_shot_prompt = FewShotPromptTemplate(
      example_selector=example_selector,
      example_prompt=example_prompt,
      prefix=system_prefix,
      suffix="",
      input_variables=["input"],
    )

    # print(few_shot_prompt)
  except Exception as e:
    # print("Fall_back")
    few_shot_prompt = FewShotPromptTemplate(
      examples=fallback_example,
      example_prompt=example_prompt,
      prefix=system_prefix,
      suffix="",
      input_variables=["input"],
    )

  prompt_val = few_shot_prompt.format(input=user_query)

  # print(prompt_val)

  system_unique_name_prompt = f"""
  If you need to filter on a proper noun, you must Always first look up the filter value using the "search_proper_nouns" tool!
  You have to access to the following tables: {', '.join(db.get_usable_table_names())}

  If the question doess not seem related to the database, just return "I don't know" as the answer.
  """
  final_prompt = prompt_val + "\n" + system_unique_name_prompt

  full_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", final_prompt),
          ("human", "{input}"),
          MessagesPlaceholder("agent_scratchpad"),
      ]
  )

  # print(final_prompt)

  agent = create_sql_agent(
      llm=llm,
      db = db,
      max_iterations=40,
      extra_tools=[retriever_tool],
      prompt=full_prompt,
      agent_type="openai-tools",
      verbose=True
  )

  result = agent.invoke({"input": user_query})
  return result['output']

In [77]:
result = get_answer("Giá của sản phẩm ' Lốp (Vỏ) xe đạp Deli'")



> Entering new SQL Agent Executor chain...

Invoking: `search_proper_nouns` with `{'query': 'Lốp (Vỏ) xe đạp Deli'}`


Bộ quần áo thể thao đi xe đạp SOUDAL LOTTO

Bộ quần áo thể thao đi xe đạp SOUDAL LOTTO

Bộ quần áo thể thao đi xe đạp SOUDAL LOTTO

Bộ quần áo thể thao đi xe đạp SOUDAL LOTTO

Xe Đạp

Xe Đạp

Xe Đạp

Xe Đạp

Tay cầm xe đạp TPR thiết kế cao su với vỏ bọc chống sốc cho xe MTB địa hình phi 2.2cm SZ-060 Mai Lee

Tay cầm xe đạp TPR thiết kế cao su với vỏ bọc chống sốc cho xe MTB địa hình phi 2.2cm SZ-060 Mai Lee

Tay cầm xe đạp TPR thiết kế cao su với vỏ bọc chống sốc cho xe MTB địa hình phi 2.2cm SZ-060 Mai Lee

Tay cầm xe đạp TPR thiết kế cao su với vỏ bọc chống sốc cho xe MTB địa hình phi 2.2cm SZ-060 Mai Lee

Thiết Bị Điện Tử

Thiết Bị Điện Tử

Thiết Bị Điện TửI am sorry, I cannot find the product 'Lốp (Vỏ) xe đạp Deli'. Would you like me to search for 'Xe Đạp'?


> Finished chain.


In [78]:
result

"I am sorry, I cannot find the product 'Lốp (Vỏ) xe đạp Deli'. Would you like me to search for 'Xe Đạp'?\n"

## UI

In [79]:
def chat(user_message, history):
  if history is None:
    history = []

  message = history.copy()

  message.append({"role": "user", "content": user_message})

  result = get_answer(str(message))

  bot_reply = result

  history.append({"role": "user", "content": user_message})
  history.append({"role": "assistant", "content": bot_reply})

  return "", history

In [80]:
import gradio as gr


In [81]:
with gr.Blocks() as demo:
  gr.Markdown("Chatbot SQL")
  chatbot = gr.Chatbot(type = "messages")
  with gr.Row():
    txt = gr.Textbox(
        show_label=False,
        placeholder="Nhập tin nhắn...",
    )
  txt.submit(chat, [txt, chatbot], [txt, chatbot])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f19dc5934908c04fbf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
